<a href="https://colab.research.google.com/github/Pavun-KumarCH/Agentic-RAG-Systems/blob/main/Agentic_RAG_llama_index_cloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title requirements
%pip install -U --q llama-index llama-parse llama-hub
%pip install --q llama-index-vector-stores-redis
%pip install --q llama-index-storage-docstore-redis
%pip install --q llama-index-storage-chat-store-redis
%pip install --q llama-index-llms-cohere
%pip install --q llama-index-embeddings-cohere
%pip install --q llama-index-embeddings-huggingface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.9/103.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.8/173.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.7/383.7 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
#@title Redis Local Vector Store Setup
%%sh
curl -fsSL https://packages.redis.io/gpg | sudo gpg --dearmor -o /usr/share/keyrings/redis-archive-keyring.gpg
echo "deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb $(lsb_release -cs) main" | sudo tee /etc/apt/sources.list.d/redis.list
sudo apt-get update  > /dev/null 2>&1
sudo apt-get install redis-stack-server  > /dev/null 2>&1
redis-stack-server --daemonize yes

deb [signed-by=/usr/share/keyrings/redis-archive-keyring.gpg] https://packages.redis.io/deb jammy main
Starting redis-stack-server, database path /var/lib/redis-stack


In [4]:
#@title Environment Variables and Redis Config
import os
from google.colab import userdata

# Replace with your actual keys
os.environ["LLAMA_CLOUD_API_KEY"] = userdata.get("LLAMA_CLOUD_API_KEY")
os.environ["COHERE_API_KEY"] = userdata.get("COHERE_API_KEY")

# Redis connection settings
REDIS_HOST = "localhost"
REDIS_PORT = 6379
REDIS_PASSWORD = ""  # Use if necessary

In [5]:
#@title Enable Asynchronous Code in Colab / Jupyter
import nest_asyncio
nest_asyncio.apply()

* Since LlamaIndex and some other modules might use asynchronous programming (e.g., network requests or embedding generation), this ensures the Jupyter environment can handle such async tasks without running into deadlocks or blocking issues.

In [6]:
#@title Download and Prepare Dataset
!mkdir -p 'data/'
!wget 'https://raw.githubusercontent.com/redis-developer/LLM-Document-Chat/main/docs/2022-chevrolet-colorado-ebrochure.pdf' -O 'data/2022-chevrolet-colorado-ebrochure.pdf'

--2024-10-17 16:53:11--  https://raw.githubusercontent.com/redis-developer/LLM-Document-Chat/main/docs/2022-chevrolet-colorado-ebrochure.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3566101 (3.4M) [application/octet-stream]
Saving to: ‘data/2022-chevrolet-colorado-ebrochure.pdf’

data/2022-chevrolet 100%[===================>]   3.40M  --.-KB/s    in 0.1s    

2024-10-17 16:53:12 (32.4 MB/s) - ‘data/2022-chevrolet-colorado-ebrochure.pdf’ saved [3566101/3566101]



In [9]:
#@title PDF Document Parsing and Loading
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader

# Initialize the PDF parser
parser = LlamaParse(
    result_type="markdown"  # You can choose between "markdown" and "text"
)

# Specify file extractor for PDFs
file_extractor = {".pdf": parser}

# Load the document using SimpleDirectoryReader
reader = SimpleDirectoryReader("./data", file_extractor=file_extractor)
documents = reader.load_data()

17:06:12 httpx INFO   HTTP Request: POST https://api.cloud.llamaindex.ai/api/parsing/upload "HTTP/1.1 200 OK"
Started parsing the file under job_id 93a889d6-480a-4fd3-a35e-fc332d8efba5
17:06:13 httpx INFO   HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/93a889d6-480a-4fd3-a35e-fc332d8efba5 "HTTP/1.1 200 OK"
17:06:15 httpx INFO   HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/93a889d6-480a-4fd3-a35e-fc332d8efba5 "HTTP/1.1 200 OK"
17:06:17 httpx INFO   HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/93a889d6-480a-4fd3-a35e-fc332d8efba5 "HTTP/1.1 200 OK"
17:06:20 httpx INFO   HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/93a889d6-480a-4fd3-a35e-fc332d8efba5 "HTTP/1.1 200 OK"
17:06:22 httpx INFO   HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/93a889d6-480a-4fd3-a35e-fc332d8efba5 "HTTP/1.1 200 OK"
17:06:22 httpx INFO   HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/93a889d6-480

**This module parses and processes the downloaded PDF document:**

1. It uses LlamaParse to initialize a PDF parser that converts the document to a  
readable format like markdown or plain text.
2. The SimpleDirectoryReader reads the document from the data/ folder, extracts the data using the file extractor, and loads it into memory for further processing (e.g., embedding generation, querying).
* This prepares the document for further tasks such as generating embeddings or retrieving information.

In [60]:
#@title Embeddings setup
from llama_index.embeddings.cohere import CohereEmbedding
from llama_index.core.ingestion import (
    DocstoreStrategy,
    IngestionPipeline,
    IngestionCache,
)
from llama_index.storage.kvstore.redis import RedisKVStore as RedisCache
from llama_index.storage.docstore.redis import RedisDocumentStore
from llama_index.core.node_parser import SentenceSplitter
from llama_index.vector_stores.redis import RedisVectorStore

from redisvl.schema import IndexSchema

# Ensure the API key is set
COHERE_API_KEY = os.getenv("COHERE_API_KEY")  # Fetch from environment variables

# Initialize the embedding model with the API key
embed_model = CohereEmbedding(input_type="search_document", api_key=COHERE_API_KEY)  # input_type="search_document" indicates that the embeddings will be optimized for document search tasks.

custom_schema = IndexSchema.from_dict(
    {
        "index": {
            "name": "chevy-colorado",
            "prefix": "pdf:chunk",
            "key_separator": ":"
          },
        # Customize fields that are indexed
        "fields": [
            # Required fields for LlamaIndex
            {"type": "tag", "name": "id"},
            {"type": "tag", "name": "doc_id"},
            {"type": "text", "name": "text"},
            # Custom vector field for Cohere embeddings
            {
                "type": "vector",
                "name": "vector",
                "attrs": {
                    "dims": 1024,
                    "algorithm": "hnsw",
                    "distance_metric": "cosine",
                },
            },
        ],
    }
)

**This module is essential for the embedding and vector storage pipeline:**

* It connects to Cohere to generate embeddings.
* Uses Redis to store document chunks and embeddings efficiently, enabling fast vector-based searches.
* The schema is customized for a specific document (the Chevrolet Colorado brochure in this case), allowing for document search and retrieval based on text similarity and embeddings.

In [11]:
#@title Vectorstore (IngestionPipeline)

vector_index_pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(),  # Split the documents into sentences
        embed_model,  # Embed the split sentences using Cohere embeddings
    ],
    # Redis document store to store the original documents
    docstore=RedisDocumentStore.from_host_and_port(
        REDIS_HOST, REDIS_PORT, namespace="doc-store"
    ),
    # Redis vector store for storing vectors (from embeddings) in Redis
    vector_store=RedisVectorStore(
        schema=custom_schema,  # The custom schema defined earlier
        redis_url=f"redis://{REDIS_HOST}:{REDIS_PORT}",
    ),
    # Cache to avoid reprocessing documents
    cache=IngestionCache(
        cache=RedisCache.from_host_and_port(REDIS_HOST, REDIS_PORT),
        collection="doc-cache",  # A namespace for caching document chunks
    ),
    # Upsert strategy ensures documents are updated if they already exist
    docstore_strategy=DocstoreStrategy.UPSERTS,
)


####**IngestionCache:**

* Prevents reprocessing of already ingested documents, improving efficiency by caching previously processed document chunks.
* RedisCache: Stores the cache in Redis, ensuring that the cache persists across sessions.
* collection="doc-cache": A namespace in Redis used to manage the cached data for documents, ensuring that caching and retrieval processes are organized.

####**DocstoreStrategy.UPSERTS:**
* This strategy ensures that if a document already exists in the store, it will be updated with any new changes rather than duplicating the data.
* This is useful in iterative processes where documents might be updated or new information is added without creating redundant records in the database.


In [61]:
#vector_index_pipeline.run(documents=documents)
vector_index_pipeline.run(documents=documents, show_progress=True)

[]

# Building the ReAct Agent
In this section we define a ReAct agent that will perform RAG over a PDF document using the Cohere command-r-plus language model.


**command-r-plus**
* This specifies which model to use from Cohere's offerings. The command-r-plus
model is optimized for reasoning and language generation tasks. It can handle more complex commands and responses, making it ideal for answering questions, generating responses, or processing natural language inputs.


**ReAct**
* A agent based on a query engine over your data. For each chat interaction, the agent enter a ReAct loop:

decide whether to use the query engine tool and come up with appropriate input
(optional) use the query engine tool and observe its output
decide whether to repeat or give final response
## **Agent Setup**
Below we define our LLM, embedding model, and a chat memory layer backed by Redis for conversation history and context.

We define both a vector index (for semantic search) and summary index (for summarization) for the document -- used as tools by the agent.

In [51]:
#@title Setup Cohere as the base embedding model and LLM
from llama_index.llms.cohere import Cohere
from llama_index.core import Settings


# Make sure to set your API keys before this step
COHERE_API_KEY = os.getenv("COHERE_API_KEY")  # Get the API key from environment variables

# Initialize the Cohere embedding model with the API key
embed_model = CohereEmbedding(input_type="search_query", api_key=COHERE_API_KEY)

# Initialize the LLM with the desired model & inference parameters
llm = Cohere(
    model="command-r-plus",  # Model version to use
    temperature=0.7,         # Controls randomness
    # max_tokens = 1000,        # Max tokens for the generation
    system_prompt = "You are a helpful assistant and a generative expert."  # Optional system prompt
  )

# Set the embedding and LLM settings
Settings.llm = llm
Settings.embed_model = embed_model

In [37]:
#@title Set up memory for the Agent (Buffer Memory For Caching)
from llama_index.storage.chat_store.redis import RedisChatStore
from llama_index.core.memory import ChatMemoryBuffer

# Build chat memory backed by Redis
chat_memory = ChatMemoryBuffer.from_defaults(
    token_limit=3000,
    chat_store=RedisChatStore(redis_url=f"redis://{REDIS_HOST}:{REDIS_PORT}", ttl=300),
    chat_store_key="user_1")

* This module initializes a chat memory system that uses Redis for caching conversation data. It allows an agent to retain context across interactions, enhancing its ability to respond appropriately.

* ChatMemoryBuffer: This class provides a buffer for storing chat memory, which can help retain context during conversations and improve the agent's performance in generating relevant responses.

* token_limit=3000: This sets the maximum number of tokens (words and punctuation) that can be stored in the chat memory buffer. This limit helps manage memory usage and ensures the system can operate efficiently.

* ttl=300: This sets a time-to-live (TTL) for the stored chat messages, meaning that chat entries will expire and be removed from Redis after 300 seconds (5 minutes) if not accessed. This helps manage memory and ensures that outdated conversations are cleared away.


In [39]:
from llama_index.core.agent import ReActAgent
from llama_index.core import SummaryIndex, VectorStoreIndex
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.node_parser import SentenceSplitter

import pickle


async def build_doc_agent(doc):
    # ID will be base + parent
    file_name = doc.metadata["file_name"]
    file_path = f"./data/{file_name}"
    file_id = file_name.replace("-", "_").strip(".pdf")
    print(file_id)

    # Run ingestion
    vector_index_pipeline.run(documents=[doc], show_progress=True)

    # Grab the nodes
    node_parser = SentenceSplitter()
    nodes = node_parser.get_nodes_from_documents([doc])

    # Define vector index and query engine
    vector_index = VectorStoreIndex.from_vector_store(
        vector_index_pipeline.vector_store
    )
    vector_query_engine = vector_index.as_query_engine()

    # Build summary index and extract a summary
    summary_index = SummaryIndex(nodes)
    summary_query_engine = summary_index.as_query_engine(
        response_mode="tree_summarize"
    )
    summary_out_path = f"./data/{file_name}_summary.pkl"
    summary = str(
        await summary_query_engine.aquery(
            "Extract a concise 1-2 line summary of this document"
        )
    )
    pickle.dump(summary, open(summary_out_path, "wb"))

    # Define agent tools
    query_engine_tools = [
        QueryEngineTool(
            query_engine=vector_query_engine,
            metadata=ToolMetadata(
                name=f"vector_tool_{file_id}",
                description=f"Useful for questions related to specific facts about the chevy colorado",
            ),
        )
    ]

    # Build ReAct agent
    agent = ReActAgent.from_tools(
      query_engine_tools,
      llm=llm,
      verbose=True,
      memory=chat_memory, # Assigns the chat memory buffer to retain context across interactions.
      context=f"""\
                You are a specialized, trustworthy, helpful, and technical customer support agent designed to answer queries about the Chevy Colorado 2022 vehicle.
                Use the available tools provided when answering a question. Do NOT just blindly make things up about the car unless it is grounded by the retrieved sources.\
                """)

    return agent, summary

In [45]:
from IPython.display import Markdown, display
agent, doc_summary = await build_doc_agent(documents[0])

2022_chevrolet_colorado_ebrochure
18:26:58 httpx INFO   HTTP Request: POST https://api.cohere.com/v1/chat "HTTP/1.1 200 OK"
18:26:58 llama_index.core.agent.react.formatter WARNING   ReActChatFormatter.from_context is deprecated, please use `from_defaults` instead.


* The await keyword indicates that this is an asynchronous function call, meaning it will pause execution until the build_doc_agent function completes and returns its results.

In [46]:
doc_summary

'A file path for a 2022 Chevrolet Colorado ebrochure.'

In [47]:
response = agent.chat("What is the seating capacity of the vehicle?")
Markdown(str(response))

> Running step 91cdfa18-ecd4-4c06-b4ea-88a9922a17c8. Step input: What is the seating capacity of the vehicle?
18:26:59 httpx INFO   HTTP Request: POST https://api.cohere.com/v1/chat "HTTP/1.1 200 OK"
Thought: (Implicit) I can answer without any more tools!
Answer: Unfortunately, I am unable to locate information regarding the seating capacity of the 2022 Chevy Colorado.


Unfortunately, I am unable to locate information regarding the seating capacity of the 2022 Chevy Colorado.

In [48]:
response = agent.chat("What is the towing capacity?")
Markdown(str(response))

> Running step 10c3988c-7fbc-4e17-8f12-55798fbba517. Step input: What is the towing capacity?
18:27:17 httpx INFO   HTTP Request: POST https://api.cohere.com/v1/chat "HTTP/1.1 200 OK"
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: vector_tool_2022_chevrolet_colorado_ebrochure
Action Input: {'input': 'What is the towing capacity of the 2022 Chevy Colorado?'}
18:27:17 httpx INFO   HTTP Request: POST https://api.cohere.com/v1/embed "HTTP/1.1 200 OK"
18:27:17 llama_index.vector_stores.redis.base INFO   Querying index chevy-colorado with query *=>[KNN 2 @vector $vector AS vector_distance] RETURN 5 id doc_id text _node_content vector_distance SORTBY vector_distance ASC DIALECT 2 LIMIT 0 2
18:27:17 llama_index.vector_stores.redis.base INFO   Found 2 results for query with id ['pdf:chunk:61fec260-76d1-4f3f-a76b-a65f5376d38b', 'pdf:chunk:21a78415-32c4-4d86-8ecc-25965271f16e']
18:27:24 httpx INFO   HTTP Request: POST https://ap

The 2022 Chevy Colorado with the Duramax® 2.8L Turbo-Diesel engine can tow up to 7,700 lbs. The towing capacity can vary based on the configuration and additional packages chosen.

In [49]:
response = agent.chat("Is there a trailer hitch on the back of the truck?")
Markdown(str(response))

> Running step 0fd02504-f696-47a4-8d25-86328fb07f61. Step input: Is there a trailer hitch on the back of the truck?
18:27:42 httpx INFO   HTTP Request: POST https://api.cohere.com/v1/chat "HTTP/1.1 200 OK"
Thought: I need to use a tool to help me answer the question.
Action: vector_tool_2022_chevrolet_colorado_ebrochure
Action Input: {'input': 'Is there a trailer hitch on the back of the truck?'}
18:27:42 httpx INFO   HTTP Request: POST https://api.cohere.com/v1/embed "HTTP/1.1 200 OK"
18:27:42 llama_index.vector_stores.redis.base INFO   Querying index chevy-colorado with query *=>[KNN 2 @vector $vector AS vector_distance] RETURN 5 id doc_id text _node_content vector_distance SORTBY vector_distance ASC DIALECT 2 LIMIT 0 2
18:27:42 llama_index.vector_stores.redis.base INFO   Found 2 results for query with id ['pdf:chunk:afae7a24-0d2c-4e5c-9f20-571dc5862c40', 'pdf:chunk:993969bd-ac60-4c75-9a03-5d12654976d6']
18:27:44 httpx INFO   HTTP Request: POST https://api.cohere.com/v1/chat "HTTP/1.

Yes, the 2022 Chevy Colorado is equipped with a trailer hitch at the back of the truck, enabling you to tow trailers or other equipment.

In [50]:
response = agent.chat("Tell me about the pros and cons of this truck.")
Markdown(str(response))

> Running step a9bc9e0a-d581-43ef-b4da-46e9239454d9. Step input: Tell me about the pros and cons of this truck.
18:27:52 httpx INFO   HTTP Request: POST https://api.cohere.com/v1/chat "HTTP/1.1 200 OK"
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: vector_tool_2022_chevrolet_colorado_ebrochure
Action Input: {'input': 'pros and cons'}
18:27:52 httpx INFO   HTTP Request: POST https://api.cohere.com/v1/embed "HTTP/1.1 200 OK"
18:27:52 llama_index.vector_stores.redis.base INFO   Querying index chevy-colorado with query *=>[KNN 2 @vector $vector AS vector_distance] RETURN 5 id doc_id text _node_content vector_distance SORTBY vector_distance ASC DIALECT 2 LIMIT 0 2
18:27:52 llama_index.vector_stores.redis.base INFO   Found 2 results for query with id ['pdf:chunk:8399d7bf-99e7-4410-b4d5-c7cdb7f12346', 'pdf:chunk:993969bd-ac60-4c75-9a03-5d12654976d6']
18:28:48 httpx INFO   HTTP Request: POST https://api.cohere.com/v1/chat "HT

Here are some of the pros and cons of the 2022 Chevy Colorado:

## Work Truck (WT) Pros:
- Offers a variety of cab options to choose from.
- Multiple engine options are available.
- The transfer case includes a Neutral position for dinghy towing.
- Comes with either a 6-speed or 8-speed automatic transmission.
- Features manual-adjustable mirrors.
- Front recovery hooks are included.
- Various wheel options are available.
- Air conditioning is standard.
- Equipped with the Chevrolet Infotainment 3 system and a 7-inch touch-screen.
- Rear Vision Camera enhances rear visibility.
- Includes a driver's seat power adjuster.

## Work Truck (WT) Cons:
- Only manual-adjustable mirrors are available.
- Front recovery hooks are standard, but there are no rear recovery hooks mentioned.
- There's only one option for the entertainment system.
- Only the driver's seat has a power adjuster.

## ZR2 Pros:
- Available in Extended Cab and Crew Cab Short Box configurations.
- Features a full locking front and rear differential, driver selectable.
- Equipped with a 3.6L DOHC V6 engine, offering Variable Valve Timing (VVT) and Direct Injection.
- Includes a Trailering Package for enhanced towing capabilities.
- Offers a 2-speed electronic Autotrac transfer case with Neutral for dinghy towing.
- 8-speed automatic transmission is standard.
- Heated, power-adjustable mirrors for improved visibility.
- Red recovery hooks are included.
- Various wheel options are available.
- Automatic, single-zone air conditioning.
- Upgraded Chevrolet Infotainment 3 Plus system with an 8-inch HD touch-screen.
- Remote vehicle starter system for added convenience.
- Both the driver and front passenger seats have power adjusters and power lumbar control.
- Heated, leather-wrapped steering wheel with mounted controls.

## ZR2 Cons:
- The ZR2 is not available in the Crew Cab Long Box configuration.
- There are no engine options or standard transmission choices mentioned.
- No standard air conditioning option is listed, but automatic single-zone air conditioning is included.

In [59]:
#@title Chat History
agent.memory.chat_store.get_messages("user_1")

[ChatMessage(role=<MessageRole.USER: 'user'>, content='What is the seating capacity of the vehicle?', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='Apologies, but I could not find any information about the seating capacity of the 2022 Chevy Colorado.', additional_kwargs={}),
 ChatMessage(role=<MessageRole.USER: 'user'>, content='What is the towing capacity?', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='The maximum towing capacity depends on the model and engine type of the 2022 Chevy Colorado. For example, the Extended Cab model can tow up to 7,000 lbs, while the Crew Cab Short Box WT and Crew Cab Long Box WT have maximum trailering weight ratings of 7,000 lbs and 7,600 lbs, respectively.', additional_kwargs={}),
 ChatMessage(role=<MessageRole.USER: 'user'>, content='What is the seating capacity of the vehicle?', additional_kwargs={}),
 ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='A

In [58]:
#@title Retrieve Chat messages
messages = agent.memory.chat_store.get_messages("user_1")

# Display the messages
for message in messages:
    print(f"{message.role.value}: {message.content}")  # Access using dictionary-like access

user: What is the seating capacity of the vehicle?
assistant: Apologies, but I could not find any information about the seating capacity of the 2022 Chevy Colorado.
user: What is the towing capacity?
assistant: The maximum towing capacity depends on the model and engine type of the 2022 Chevy Colorado. For example, the Extended Cab model can tow up to 7,000 lbs, while the Crew Cab Short Box WT and Crew Cab Long Box WT have maximum trailering weight ratings of 7,000 lbs and 7,600 lbs, respectively.
user: What is the seating capacity of the vehicle?
assistant: Apologies, but I still cannot find the seating capacity of the 2022 Chevy Colorado.
user: What is the seating capacity of the vehicle?
assistant: Unfortunately, I am unable to locate information regarding the seating capacity of the 2022 Chevy Colorado.
user: What is the towing capacity?
assistant: The 2022 Chevy Colorado with the Duramax® 2.8L Turbo-Diesel engine can tow up to 7,700 lbs. The towing capacity can vary based on the c

## Incorporating Semantic Caching
We can also take advantage of frequently asked questions (live or prefetched) in order to improve response times.

In [24]:
from redisvl.extensions.llmcache import SemanticCache
from redisvl.utils.vectorize import HFTextVectorizer

emb = HFTextVectorizer(model="BAAI/bge-small-en-v1.5")

cache = SemanticCache(
    name="chevy_cache",
    prefix="cache",
    distance_threshold=0.2,
    ttl=60,
    vectorizer=emb
)

17:11:31 sentence_transformers.SentenceTransformer INFO   Use pytorch device_name: cpu
17:11:31 sentence_transformers.SentenceTransformer INFO   Load pretrained SentenceTransformer: BAAI/bge-small-en-v1.5


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
def invoke_agent(prompt: str) -> str:
    if cached_result := cache.check(prompt=prompt):
        response = cached_result[0]['response']
        return response
    response = agent.chat(prompt)
    # cache.store(prompt=prompt, response=response.response)
    return response.response

Now we can perform a simple test with our agent and semantic caching enabled.



In [ ]:
invoke_agent("How many doors does the truck have?")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

> Running step f21c7666-1bf0-459b-8288-599c2645a62f. Step input: How many doors does the truck have?
21:02:49 httpx INFO   HTTP Request: POST https://api.cohere.com/v1/chat "HTTP/1.1 200 OK"
Thought: (Implicit) I can answer without any more tools!
Answer: The 2022 Chevy Colorado is typically offered in two body styles: an Extended Cab with two doors and a Crew Cab with four doors. The Extended Cab features a 2+2 seating configuration, providing additional cargo space, while the Crew Cab offers a more traditional 4-door design with ample room for passengers.


'The 2022 Chevy Colorado is typically offered in two body styles: an Extended Cab with two doors and a Crew Cab with four doors. The Extended Cab features a 2+2 seating configuration, providing additional cargo space, while the Crew Cab offers a more traditional 4-door design with ample room for passengers.'

In [ ]:
invoke_agent("How many passenger doors are on the truck?")


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

> Running step 4a4e6aee-bfda-4d82-b74d-6ee19535cdf7. Step input: How many passenger doors are on the truck?
21:02:52 httpx INFO   HTTP Request: POST https://api.cohere.com/v1/chat "HTTP/1.1 200 OK"
Thought: (Implicit) I can answer without any more tools!
Answer: The number of passenger doors depends on the cab style you choose. If you opt for the Extended Cab, there is one passenger door on the driver's side, resulting in a total of two doors (including the driver's door). On the other hand, the Crew Cab features two passenger doors, one on each side, for a total of four doors. So, depending on your preference and needs, you can choose between two or four passenger doors with the 2022 Chevy Colorado.


"The number of passenger doors depends on the cab style you choose. If you opt for the Extended Cab, there is one passenger door on the driver's side, resulting in a total of two doors (including the driver's door). On the other hand, the Crew Cab features two passenger doors, one on each side, for a total of four doors. So, depending on your preference and needs, you can choose between two or four passenger doors with the 2022 Chevy Colorado."

# Extending Semantic Caching
There are a few options for working with semantic caching in a true production setting:

1. Extract FAQs from your Knowledge Base (pdfs...). Use an LLM to help! Or use human experts. Prefetch into the cache.
2. Carefully, extract FAQs from conversation history. Prefetch in batches into the cache each day or week.

**Cache in realtime, which is tricky and requires some tuning. This should typically be done only at the USER level, not across the entire domain or application.**